# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jimal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jimal\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jimal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_responses.db')
df = pd.read_sql('messages', engine)
X = df['message']
Y = df.drop(columns=['id','message','original','genre', 'child_alone']) #drop child_alone category because it has all 0s

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    #replace all urls with urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, 'urlplaceholder')
    
    #normalize and tokenize text
    text = re.sub(r'[^a-zA-Z0-9],', ' ', text.lower())
    tokens = word_tokenize(text)
    
    #strip tokens and remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [token.strip() for token in tokens if token.strip() not in stop_words]
    
    #lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
pipeline = Pipeline([
    ('text_pipeline', TfidfVectorizer(tokenizer=tokenize)),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
for i in range(y_test.shape[1]):
    print(y_test.columns[i] + '\n' + classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print(confusion_matrix(y_test.iloc[:,i], y_pred[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
parameters = {
    'text_pipeline__max_df' : [0.5, 0.75, 1.0],
    'text_pipeline__min_df' : [1, 50, 100],
}

cv = GridSearchCV(pipeline, parameters, scoring='f1_micro', n_jobs=-1, cv=3)

cv.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('text_pipeline',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000019D94DC5840>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'text_pipeline__max_df': [0.5, 0.75, 1.0],
                         'text_pipeline__min_df': [1, 50, 100]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
y_pred = cv.best_estimator_.predict(X_test)
for i in range(y_test.shape[1]):
    print(y_test.columns[i] + '\n' + classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print(confusion_matrix(y_test.iloc[:,i], y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.69      0.41      0.52      1513
           1       0.84      0.94      0.89      5031

    accuracy                           0.82      6544
   macro avg       0.77      0.68      0.70      6544
weighted avg       0.81      0.82      0.80      6544

[[ 625  888]
 [ 279 4752]]
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5396
           1       0.83      0.52      0.64      1148

    accuracy                           0.90      6544
   macro avg       0.87      0.75      0.79      6544
weighted avg       0.89      0.90      0.89      6544

[[5277  119]
 [ 554  594]]
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6512
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6544
   macro avg       0.50      0.50      0.50      654

c:\Users\jimal\Anaconda3\envs\python36\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.64      0.13      0.21        70

    accuracy                           0.99      6544
   macro avg       0.82      0.56      0.60      6544
weighted avg       0.99      0.99      0.99      6544

[[6469    5]
 [  61    9]]
refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6326
           1       0.58      0.11      0.19       218

    accuracy                           0.97      6544
   macro avg       0.78      0.56      0.59      6544
weighted avg       0.96      0.97      0.96      6544

[[6308   18]
 [ 193   25]]
death
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      6220
           1       0.76      0.38      0.51       324

    accuracy                           0.96      6544
   macro avg       0.86      0.69      0.74 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
import pickle
with open('classifier.pkl', mode='wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.